In [1]:
import pandas as pd
import numpy as np

## Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
  - оценка 2 и меньше - низкий рейтинг
  - оценка 4 и меньше - средний рейтинг
  - оценка 4.5 и 5 - высокий рейтинг
​
Результат классификации запишите в столбец class

In [2]:
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')
links = pd.read_csv('ml-latest-small/links.csv')
tags = pd.read_csv('ml-latest-small/tags.csv')

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [5]:
mean_ratings = ratings[['movieId', 'rating']].groupby('movieId').agg({'rating':'mean'})
mean_ratings

,rating
movieId,
1,3.872470
2,3.401869
3,3.161017
4,2.384615
5,3.267857
...,...
161944,5.000000
162376,4.500000
162542,5.000000


In [6]:
movies_with_rating = (movies.merge(mean_ratings, on='movieId', how='outer').
                      rename(columns={'rating' : 'mean_rating'}))

movies_with_rating.head()

,movieId,title,genres,mean_rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.872470
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.401869
2,3,Grumpier Old Men (1995),Comedy|Romance,3.161017
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.384615
4,5,Father of the Bride Part II (1995),Comedy,3.267857


In [7]:
# Пробую предусмотреть все случаи

def define_class(rating):
    if not pd.isna(rating):
        if 0 <= rating <= 2:
            return 'Low rating'
        elif 2 < rating <=4:
            return 'Medium rating'
        elif 4 < rating <= 5:
            return 'High rating'
        else:
            return "Wrong rating"
    else:
        return rating

In [8]:
movies_with_rating['class'] = movies_with_rating.mean_rating.apply(define_class)
movies_with_rating.tail(10)

,movieId,title,genres,mean_rating,class
9115,161830,Body (2015),Drama|Horror|Thriller,1.0,Low rating
9116,161918,Sharknado 4: The 4th Awakens (2016),Action|Adventure|Horror|Sci-Fi,1.5,Low rating
9117,161944,The Last Brickmaker in America (2001),Drama,5.0,High rating
9118,162376,Stranger Things,Drama,4.5,High rating
9119,162542,Rustom (2016),Romance|Thriller,5.0,High rating
9120,162672,Mohenjo Daro (2016),Adventure|Drama|Romance,3.0,Medium rating
9121,163056,Shin Godzilla (2016),Action|Adventure|Fantasy|Sci-Fi,NaN,NaN
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,Documentary,5.0,High rating
9123,164977,The Gay Desperado (1936),Comedy,NaN,NaN
9124,164979,"Women of '69, Unboxed",Documentary,NaN,NaN


## Задание 2
Используем файл keywords.csv.
​
Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.
​
Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:
​
geo_data = {
​
    'Центр': ['москва', 'тула', 'ярославль'],
​
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
​
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
​
}
​
Результат классификации запишите в отдельный столбец region.

In [42]:
keywords = pd.read_csv('keywords.csv')

geo_data = {'Центр': ['москва', 'тула', 'ярославль'], 
            'Северо-Запад': ['петербург', 'псков', 'мурманск'],
            'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}

keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [41]:
def check_city(keyword):
    for word in keyword.split(' '):
        for region in geo_data.keys():
            if word in geo_data[region]:
                return region
    return 'undefined'
check_city('сахалин asd')

'Дальний Восток'

In [43]:
keywords['region'] = keywords.keyword.apply(check_city)
keywords.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [44]:
keywords.query("region in @geo_data.keys()").head()

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
1236,банк санкт петербург,174375,Северо-Запад


## Задание 3 (бонусное)
Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.
​
При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:
​
1. В переменную years запишите список из всех годов с 1950 по 2010.
​
2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
  - для каждой строки пройдите по всем годам списка years
  - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
  - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
​
3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'
​
4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга


In [48]:
movies_with_rating.head()

,movieId,title,genres,mean_rating,class
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.872470,Medium rating
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.401869,Medium rating
2,3,Grumpier Old Men (1995),Comedy|Romance,3.161017,Medium rating
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.384615,Medium rating
4,5,Father of the Bride Part II (1995),Comedy,3.267857,Medium rating


In [154]:
def extract_year(title):
    pattern = r'(\d{4})'
    year = re.findall(pattern, title)
    if year:
        for item in year: 
            if 1950 <= int(item) <= 2010:
                return int(item)
        return 1900
    
movies_with_rating['year'] = (
    movies_with_rating.title.
    apply(extract_year).
    fillna(1900).
    apply(round)
)

movies_with_rating.head()

,movieId,title,genres,mean_rating,class,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.872470,Medium rating,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.401869,Medium rating,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,3.161017,Medium rating,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.384615,Medium rating,1995
4,5,Father of the Bride Part II (1995),Comedy,3.267857,Medium rating,1995


In [152]:
mean_rating_by_years = (
    movies_with_rating[['mean_rating', 'year']].
    groupby('year').
    agg('mean').
    reset_index().
    sort_values('mean_rating', ascending=False)
)

In [153]:
mean_rating_by_years

,year,mean_rating
2,1951,3.906653
3,1952,3.802318
13,1962,3.772962
4,1953,3.738878
8,1957,3.693322
...,...,...
51,2000,3.187543
48,1997,3.183169
41,1990,3.162096
54,2003,3.154081


In [163]:
#проверка выполнения условия 
mean_rating_by_years[mean_rating_by_years.year.between(1950, 2010)]

,year,mean_rating
2,1951,3.906653
3,1952,3.802318
13,1962,3.772962
4,1953,3.738878
8,1957,3.693322
...,...,...
51,2000,3.187543
48,1997,3.183169
41,1990,3.162096
54,2003,3.154081


In [164]:
mean_rating_by_years[~mean_rating_by_years.year.between(1950, 2010)]

,year,mean_rating
0,1900,3.245485
